In [4]:
%pip install pdfplumber chromadb sentence-transformers langchain

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pdfplumber
from pathlib import Path

pdf_path = Path("../data/LoisduJeu2024_25.pdf")
raw_txt_path = Path("../data/loi_jeu_raw.txt")

all_text = ""

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            all_text += text + "\n"

raw_txt_path.write_text(all_text, encoding="utf-8")
raw_txt_path


WindowsPath('../data/loi_jeu_raw.txt')

In [2]:
import re

def clean_text(text):
    # Supprimer numéros de page
    text = re.sub(r"\b[Pp]age\s+\d+\b", " ", text)
    text = re.sub(r"\n\s*\d+\s*\n", " ", text)

    # Supprimer lignes trop courtes (en-têtes)
    lines = [ln.strip() for ln in text.split("\n")]
    lines = [ln for ln in lines if len(ln) > 10]

    text = " ".join(lines)

    # Remplacer espaces multiples
    text = re.sub(r"\s+", " ", text)
    
    return text.strip()

raw = raw_txt_path.read_text(encoding="utf-8")
cleaned = clean_text(raw)

cleaned_path = Path("../data/loi_jeu_clean.txt")
cleaned_path.write_text(cleaned, encoding="utf-8")

cleaned[:1000]  # aperçu


'24/25 52/4202 The International Football Association Board Münstergasse 9, 8001 Zurich, Suisse Tél. : +41 (0)44 245 1886, Fax : +41 (0)44 245 1887 www.theifab.com Reproduction ou traduction complètes ou partielles seulement avec l’autorisation spéciale de l’IFAB. En vigueur depuis le 1er juillet 2024 Lois 2024/25 Table des matières Notes et modifications 16 Remarques relatives aux Lois du Jeu 17 Modifications générales 20 Directives pour les exclusions temporaires 24 Directives pour les remplacements libres 30 Protocole relatif aux remplacements permanents supplémentaires pour commotion cérébrale 32 Lois du Jeu 2024/25 37 1 Terrain 39 2 Ballon 49 3 Joueurs 53 4 Équipement des joueurs 61 5 Arbitre 69 6 Autres arbitres 81 7 Durée d’un match 89 8 Coup d’envoi et reprise du jeu 93 9 Ballon en jeu et hors du jeu 97 10 Issue d’un match 99 11 Hors-jeu 105 12 Fautes et incorrections 111 13 Coups francs 127 14 Penalty 131 15 Rentrée de touche 137 16 Coup de pied de but 141 17 Corner 145 Protoc

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

chunks = splitter.split_text(cleaned)
len(chunks)

c:\Users\MSI\Desktop\projetChatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


355

In [4]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.PersistentClient(path="../vectors/chroma_db")

embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

collection = chroma_client.get_or_create_collection(
    name="fifa_rules",
    embedding_function=embedding_fn
)

ids = [f"chunk_{i}" for i in range(len(chunks))]

collection.add(
    documents=chunks,
    ids=ids
)

print("ChromaDB créée")


ChromaDB créée


In [5]:
query = "quelle est la règle du hors jeu ?"

results = collection.query(
    query_texts=[query],
    n_results=3
)
results['documents'][0]


['être averti • Un gardien de but exclu doit être remplacé par un joueur autorisé. • Un joueur, autre que le gardien de but, qui n’est pas en mesure de continuer ne peut pas être remplacé. • L’arbitre n’arrêtera pas le match définitivement si une équipe se retrouve à moins de sept joueurs. Lois du Jeu 2024/25 | Loi 10 | Issue d’un match 103 Loi Hors-jeu 1. Position de hors-jeu Être en position de hors-jeu n’est pas une infraction. Un joueur est en position de hors-jeu si : • n’importe quelle partie de la tête, du tronc ou des jambes se trouve dans la moitié de terrain adverse (ligne médiane non comprise) ; et • n’importe quelle partie de la tête, du tronc ou des jambes se trouve plus près de la ligne de but adverse que le ballon et l’avant-dernier adversaire. Les mains et bras de tous les',
 'jouer le ballon, ou avant de disputer le ballon à un adversaire, la faute est sanctionnée car elle s’est produite avant l’infraction de hors-jeu ; • une faute est commise contre un joueur en posit